# importing packages for web scrapping wiki table

In [1]:
import requests
import lxml.html as lh

from bs4 import BeautifulSoup as soup

import pandas as pd

In [2]:
my_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(my_url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [3]:
len(tr_elements)

293

In [4]:
# first row
tr_elements[0].text_content()

'\nPostcode\nBorough\nNeighbourhood\n'

In [5]:
# last row
tr_elements[287].text_content()

'\nM9Z\nNot assigned\nNot assigned\n'

# creating dataframe by iteratively list appending

In [6]:
col = []
i = 0

for j in range(288):
    for t in tr_elements[j]:
        i+= 1
        info = t.text_content()
        col.append(info)

In [7]:
pst = []
bor = []
nei = []

n = 288

for j in range(1,n):
    pst.append(col[3*j])
    bor.append(col[3*j+1])
    nei.append(col[3*j+2])

for i in range(len(nei)):
    nei[i] = nei[i].replace('\n','') # removing the \n 

In [8]:
Dict = {'Postcode':pst, 'Borough':bor, 'Neighbourhood':nei }
df = pd.DataFrame(Dict)

# the first dataframe that we webscrap from wiki page

In [9]:
df.shape

(287, 3)

In [10]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
df.tail()

,Postcode,Borough,Neighbourhood
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor
286,M9Z,Not assigned,Not assigned


# the second dataframe that filter out the rows with Borough = Not assigned

In [12]:
df2 = df[df['Borough'] != 'Not assigned']

In [13]:
df2.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


# generating ref list to build unique postcode for neighbourhood

In [17]:
df2['ref'] = df2['Postcode'] +','+ df2['Borough']  # using concatenation for ref on postcode and borough

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
ref_list = list(df2['ref'].unique()) # unique ref_list

In [24]:
df2[df2['ref'] == ref_list[3]]

,Postcode,Borough,Neighbourhood,ref
5,M6A,North York,Lawrence Heights,"M6A,North York"
6,M6A,North York,Lawrence Manor,"M6A,North York"


In [25]:
nei_list = []
for i in range(len(ref_list)):
    nei_list.append(list(df2[df2['ref'] == ref_list[i]].iloc[:,2]))

In [26]:
# coverting list of list to list of string
nei_list = list(map(','.join, nei_list))

In [27]:
nei_list[0:5]

['Parkwoods',
 'Victoria Village',
 'Harbourfront',
 'Lawrence Heights,Lawrence Manor',
 "Queen's Park"]

In [28]:
df2.head()

,Postcode,Borough,Neighbourhood,ref
2,M3A,North York,Parkwoods,"M3A,North York"
3,M4A,North York,Victoria Village,"M4A,North York"
4,M5A,Downtown Toronto,Harbourfront,"M5A,Downtown Toronto"
5,M6A,North York,Lawrence Heights,"M6A,North York"
6,M6A,North York,Lawrence Manor,"M6A,North York"


# the third dataframe for unique postcode with multiple neighbourhood

In [29]:
df3 = pd.DataFrame({'ref':ref_list,'Neighbourhood':nei_list })

In [30]:
df3.head()

,ref,Neighbourhood
0,"M3A,North York",Parkwoods
1,"M4A,North York",Victoria Village
2,"M5A,Downtown Toronto",Harbourfront
3,"M6A,North York","Lawrence Heights,Lawrence Manor"
4,"M7A,Downtown Toronto",Queen's Park


In [31]:
df3[['Postcode','Borough']] = df3['ref'].str.split(',',expand=True)

In [32]:
df3.head()

,ref,Neighbourhood,Postcode,Borough
0,"M3A,North York",Parkwoods,M3A,North York
1,"M4A,North York",Victoria Village,M4A,North York
2,"M5A,Downtown Toronto",Harbourfront,M5A,Downtown Toronto
3,"M6A,North York","Lawrence Heights,Lawrence Manor",M6A,North York
4,"M7A,Downtown Toronto",Queen's Park,M7A,Downtown Toronto


In [33]:
df4 = df3[['Postcode','Borough','Neighbourhood']]

In [34]:
df4.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


# filtering for Not assigned neighbourhood with known borough

In [35]:
df4[df4['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
5,M9A,Queen's Park,Not assigned


In [36]:
# replacing the value
df4['Neighbourhood'][5] = "Queen's Park" 

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
df4[df4['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood


In [41]:
df4.head(8)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North


# final dataframe dimension

In [42]:
df4.shape

(103, 3)

In [43]:
df4.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


# merging for geocode

In [44]:
!pip install geocoder

     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011


In [45]:
import pandas as pd

In [46]:
geocode = pd.read_csv('https://cocl.us/Geospatial_data')

In [47]:
geocode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
geocode.columns = ['Postcode','Latitude','Longitude']

In [49]:
geocode.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [50]:
geocode[geocode['Postcode'] == 'M4A']

,Postcode,Latitude,Longitude
34,M4A,43.725882,-79.315572


In [51]:
df5 = df4.merge(geocode, on='Postcode', how='left')

In [52]:
df5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


# visualizing on the map

In [53]:
!pip install folium

     |████████████████████████████████| 92kB 7.7MB/s eta 0:00:011


In [54]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [55]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


In [56]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [57]:
df5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [58]:
df5['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Scarborough',
       'East York', 'Etobicoke', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [70]:
n_york_data = df5[df5['Borough'] == 'North York'].reset_index(drop=True)
n_york_data.shape

(24, 5)

In [71]:
n_york_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [72]:
address = 'North York,Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of North York,Toronto, Canada are 43.7543263, -79.44911696639593.


In [73]:
# create map of Manhattan using latitude and longitude values
map_n_york= folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(n_york_data['Latitude'], n_york_data['Longitude'], n_york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_n_york)  
    
map_n_york

# using foursquare to get information for clustering analysis

In [74]:
CLIENT_ID = 'T4YVDHPIK5IQ4YIOJ3SIVJETRM1U5A11KA0ONLGGP4QX4BRO' # your Foursquare ID
CLIENT_SECRET = '2VAH1GUOBMZHEVSAGHH2NA25VC5RZB2JWB0UB0V2N3UAGZL0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T4YVDHPIK5IQ4YIOJ3SIVJETRM1U5A11KA0ONLGGP4QX4BRO
CLIENT_SECRET:2VAH1GUOBMZHEVSAGHH2NA25VC5RZB2JWB0UB0V2N3UAGZL0


In [77]:
n_york_data.columns = ['Postcode','Borough','Neighborhood','Latitude','Longitude']

In [78]:
n_york_data.loc[0, 'Neighborhood']

'Parkwoods'

In [79]:
neighborhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = n_york_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


# calling foursquare API for neighborhood venues 

In [80]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=T4YVDHPIK5IQ4YIOJ3SIVJETRM1U5A11KA0ONLGGP4QX4BRO&client_secret=2VAH1GUOBMZHEVSAGHH2NA25VC5RZB2JWB0UB0V2N3UAGZL0&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [81]:
results = requests.get(url).json()
len(results)

2

In [82]:
type(results)

dict

In [83]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [84]:
venues = results['response']['groups'][0]['items']

In [85]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'name': 'Brookbanks Park',
   'location': {'address': 'Toronto',
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'distance': 245,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
 {'reasons': {'count': 0,
   'items': [{'

In [86]:
nearby_venues = json_normalize(venues) # flatten JSON


In [87]:
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e8d9dcdd5fbbbb6b3003c7b-0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4e8d9dcdd5fbbbb6b3003c7b,Toronto,CA,Toronto,Canada,245,"[Toronto, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197604605557...",43.751976,-79.332140,ON,Brookbanks Park,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cb11e2075ebb60cd1c4caad-1,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",4cb11e2075ebb60cd1c4caad,29 Valley Woods Road,CA,Toronto,Canada,312,"[29 Valley Woods Road, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.75197441585782...",43.751974,-79.333114,ON,Variety Store,0,[]


In [88]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [89]:
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114


In [90]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


In [91]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


In [92]:
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [93]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


# writing a function to get nearby venue for any neighborhood

In [94]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [96]:
# store nearby venue for north york

n_york_venues = getNearbyVenues(names=n_york_data['Neighborhood'],
                                   latitudes=n_york_data['Latitude'],
                                   longitudes=n_york_data['Longitude']
                                  )



Parkwoods
Victoria Village
Lawrence Heights,Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park,Don Mills South
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Fairview,Henry Farm,Oriole
Northwood Park,York University
Bayview Village
CFB Toronto,Downsview East
Silver Hills,York Mills
Downsview West
Downsview,North Park,Upwood Park
Humber Summit
Newtonbrook,Willowdale
Downsview Central
Bedford Park,Lawrence Manor East
Emery,Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [100]:
print(n_york_venues.shape)
n_york_venues.head(10)

(247, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Roots,43.718221,-79.466776,Boutique
8,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),43.719096,-79.462675,Furniture / Home Store
9,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant


In [101]:
n_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto,Downsview East",4,4,4,4,4,4
Don Mills North,4,4,4,4,4,4
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,5,5,5,5,5,5
"Downsview,North Park,Upwood Park",4,4,4,4,4,4


In [102]:
print('There are {} uniques categories.'.format(len(n_york_venues['Venue Category'].unique())))

There are 104 uniques categories.


# one hot encoding before clustering

In [103]:
# one hot encoding
n_york_onehot = pd.get_dummies(n_york_venues[['Venue Category']], prefix="", prefix_sep="")


In [104]:
# add neighborhood column back to dataframe
n_york_onehot['Neighborhood'] = n_york_venues['Neighborhood'] 


In [105]:
# move neighborhood column to the first column
fixed_columns = [n_york_onehot.columns[-1]] + list(n_york_onehot.columns[:-1])
n_york_onehot = n_york_onehot[fixed_columns]

In [106]:
n_york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [107]:
n_york_onehot.shape

(247, 105)

# calculating for percentage

In [108]:
n_york_grouped = n_york_onehot.groupby('Neighborhood').mean().reset_index()
n_york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.047619,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.047619,0.000000,0.000000,0.095238,0.000000,0.000000,0.00,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.00,0.000000,0.047619,0.000000,0.00,0.0,0.000000,0.000000,0.047619,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.047619,0.000000,0.000000,0.00,0.000000,0.047619,0.0,0.047619,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.047619,0.00,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.000000,0.00,0.041667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.0,0.041667,0.000000,0.000000,0.000000,0.000000,0.083333,0.041667,0.000000,0.00,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.083333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.041667,0.041667,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.041667,0.000000,0.0,0.083333,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.041667,0.0,0.041667,0.00,0.000000,0.00,0.0,0.041667,0.000000,0.041667,0.000000,0.083333,0.0

In [109]:
n_york_grouped.shape

(24, 105)

# generating list of top 5 venues

In [110]:
num_top_venues = 5

for hood in n_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = n_york_grouped[n_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
           venue  freq
0    Coffee Shop  0.10
1  Shopping Mall  0.05
2    Bridal Shop  0.05
3       Pharmacy  0.05
4    Pizza Place  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1           Coffee Shop  0.08
2        Sandwich Place  0.08
3  Fast Food Restaurant  0.08
4     Indian Restaurant  0.04


----CFB Toronto,Downsview East----
         venue  freq
0   Playground  0.25
1  Snack Place  0.25
2         Park  0.25
3      Airport  0.25
4        Hotel  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.25
1  Caribbean Restaurant  0.25
2                  Café  0.25
3   Japanese Restaurant  0.25
4     Accessories Store  0.00


----Downsview 

In [111]:
# function to return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [112]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = n_york_grouped['Neighborhood']

for ind in np.arange(n_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(n_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Grocery Store,Shopping Mall,Gas Station,Ice Cream Shop,Diner,Deli / Bodega,Middle Eastern Restaurant,Chinese Restaurant,Fried Chicken Joint
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park,Lawrence Manor East",Sandwich Place,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Indian Restaurant,Liquor Store,Cosmetics Shop,Pharmacy,Café,Pizza Place
3,"CFB Toronto,Downsview East",Snack Place,Airport,Park,Playground,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


# clustering neighborhood with k-mean clustering

In [119]:
# set number of clusters to be 5
kclusters = 5

n_york_grouped_clustering = n_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(n_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 0, 0, 4, 2], dtype=int32)

In [125]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [126]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Grocery Store,Shopping Mall,Gas Station,Ice Cream Shop,Diner,Deli / Bodega,Middle Eastern Restaurant,Chinese Restaurant,Fried Chicken Joint
1,0,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,0,"Bedford Park,Lawrence Manor East",Sandwich Place,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Indian Restaurant,Liquor Store,Cosmetics Shop,Pharmacy,Café,Pizza Place
3,0,"CFB Toronto,Downsview East",Snack Place,Airport,Park,Playground,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,0,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
5,2,Downsview Central,Home Service,Food Truck,Baseball Field,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
6,0,Downsview Northwest,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Liquor Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
7,0,Downsview West,Grocery Store,Park,Hotel,Bank,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
8,4,"Downsview,North Park,Upwood Park",Construction & Landscaping,Park,Bakery,Basketball Court,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
9,2,"Emery,Humberlea",Furniture / Home Store,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [129]:
n_york_merged = n_york_data

In [130]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
n_york_merged = n_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [132]:
n_york_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Park,Food & Drink Shop,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Coffee Shop,Women's Store,Diner,Concert Hall,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Accessories Store,Vietnamese Restaurant,Tea Room
3,M3B,North York,Don Mills North,43.745906,-79.352188,0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,M6B,North York,Glencairn,43.709577,-79.445073,0,Pizza Place,Park,Pub,Metro Station,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


In [134]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(n_york_merged['Latitude'], n_york_merged['Longitude'], n_york_merged['Neighborhood'], n_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# examing the cluster

In [136]:
n_york_merged.loc[n_york_merged['Cluster Labels'] == 4, n_york_merged.columns[[1] + list(range(5, n_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Park,Food & Drink Shop,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
14,North York,4,Construction & Landscaping,Park,Bakery,Basketball Court,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
22,North York,4,Park,Convenience Store,Bank,Bar,Women's Store,Dog Run,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


## Cluster 4 has charateristic that most common venue is park 

In [137]:
n_york_merged.loc[n_york_merged['Cluster Labels'] == 3, n_york_merged.columns[[1] + list(range(5, n_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,3,Cafeteria,Women's Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


## Cluster 3 does not have any clustering effect

In [138]:
n_york_merged.loc[n_york_merged['Cluster Labels'] == 2, n_york_merged.columns[[1] + list(range(5, n_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,2,Home Service,Food Truck,Baseball Field,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
19,North York,2,Furniture / Home Store,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


## Cluster 2 has charateristic that with furniture and home service

In [139]:
n_york_merged.loc[n_york_merged['Cluster Labels'] == 1, n_york_merged.columns[[1] + list(range(5, n_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,1,Piano Bar,Greek Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


## Cluster 1 does not have any clustering effect

In [140]:
n_york_merged.loc[n_york_merged['Cluster Labels'] == 0, n_york_merged.columns[[1] + list(range(5, n_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Coffee Shop,Women's Store,Diner,Concert Hall,Construction & Landscaping,Convenience Store
2,North York,0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Accessories Store,Vietnamese Restaurant,Tea Room
3,North York,0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,North York,0,Pizza Place,Park,Pub,Metro Station,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
5,North York,0,Beer Store,Coffee Shop,Asian Restaurant,Gym,Chinese Restaurant,Clothing Store,Café,Dim Sum Restaurant,Japanese Restaurant,Italian Restaurant
6,North York,0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Frozen Yogurt Shop,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Gas Station,Convenience Store
7,North York,0,Coffee Shop,Grocery Store,Shopping Mall,Gas Station,Ice Cream Shop,Diner,Deli / Bodega,Middle Eastern Restaurant,Chinese Restaurant,Fried Chicken Joint
8,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Convenience Store,Shoe Store,Juice Bar,Bakery,Japanese Restaurant,Women's Store,Food Court
9,North York,0,Coffee Shop,Furniture / Home Store,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Bar,Massage Studio,Diner,Convenience Store,Cosmetics Shop
10,North York,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


## Cluster 0 has charateristic that with stores and coffee shops